In [72]:
prods = ["CU_LT05.001"]
layers = [(prods[0], "PIXELQA"), (prods[0], "SRB1"), (prods[0], "SRB2"), (prods[0], "SRB3"), (prods[0], "SRB4"), (prods[0], "SRB5"), (prods[0], "SRB7"), (prods[0], "SRCLOUDQA")]
prodLayer = []
for l in layers:
    prodLayer.append({
        "layer": l[1],
        "product": l[0]
    })
prodLayer

[{'layer': 'PIXELQA', 'product': 'CU_LT05.001'},
 {'layer': 'SRB1', 'product': 'CU_LT05.001'},
 {'layer': 'SRB2', 'product': 'CU_LT05.001'},
 {'layer': 'SRB3', 'product': 'CU_LT05.001'},
 {'layer': 'SRB4', 'product': 'CU_LT05.001'},
 {'layer': 'SRB5', 'product': 'CU_LT05.001'},
 {'layer': 'SRB7', 'product': 'CU_LT05.001'},
 {'layer': 'SRCLOUDQA', 'product': 'CU_LT05.001'}]

In [77]:
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd

# Enter Earth Data login Credentials
username = "ZekunLin"#getpass.getpass('Earthdata Username:')
password = "P6Fn8T47"#getpass.getpass('Earthdata Password:')

api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'  # Set the AρρEEARS API to a variable

token_response = r.post('{}login'.format(api), auth=(username, password)).json() # Insert API URL, call login service, provide credentials & return json
del username, password                                                           # Remove user and password information
token_response 

{'token_type': 'Bearer',
 'token': '_S8jYB8ktSw3cXinTiVmYLHkYpuuAt1uyuIhZvCHabLLs65hgrrNc9QpiMT5eCW70D1v5dTFxTelTnwWhPfP-w',
 'expiration': '2020-10-13T05:29:57Z'}

In [78]:
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections
projs = {}
for p in projections:
    projs[p["Name"]] = p
list(projs.keys())



{'Name': 'albers_ard_conus',
 'Description': 'Landsat ARD Albers Equal Area CONUS',
 'Platforms': "['Landsat ARD']",
 'Proj4': '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs',
 'Datum': 'wgs84',
 'EPSG': '',
 'Units': 'meters',
 'GridMapping': 'albers_conical_equal_area',
 'Available': True}

In [110]:
import os
import glob
import time
tile_list = glob.glob("D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles/*.shp")
short_name = [os.path.basename(x) for x in glob.glob("D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles/*.shp")]

for i in range(len(tile_list)):
    nps = gpd.read_file(tile_list[i])
    nps = nps.to_crs("EPSG:4326")

    nps_gc = nps.to_json()
    nps_gc = json.loads(nps_gc)

    task_name = short_name[i][0:7]
    task_type = "area"
    proj = projs["geographic"]["Name"]
    outFormat = ["geotiff"]
    startDate = "01-01-1984"
    endDate = "12-31-2003"
    recurring = False

    task = {
        "task_type": task_type, # area sample task
        "task_name": task_name,
        "params": {
            "dates": [
            {
                "startDate": startDate,
                "endDate": endDate
            }],
            "layers": prodLayer,
            "output": {
                "format": {
                    "type": outFormat[0] # geotiff format
                },
                "projection": proj
            },
            "geo": nps_gc
            }
    }

    print(task)

    token = token_response['token']                      # Save login token to a variable
    head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

    task_response = r.post("{}task".format(api), json = task, headers = head).json()
    print(task_response)
    
    task_id = task_response['task_id']                                               # Set task id from request submission
    status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
    print(status_response['status'])

    write_task_id = task_id + " : " + task_name + ":" + status_response['status']
    t = open(os.path.join("D:/Zekun/Landsat_ARD/Landsat_ARD/se_ard/", "task_id_by_tile.txt"), 'a')
    t.write(write_task_id +"\n")
    t.close()

    print('orders have been submitted')


{'task_type': 'area', 'task_name': 'h18_v13', 'params': {'dates': [{'startDate': '01-01-1984', 'endDate': '12-31-2003'}], 'layers': [{'layer': 'PIXELQA', 'product': 'CU_LT05.001'}, {'layer': 'SRB1', 'product': 'CU_LT05.001'}, {'layer': 'SRB2', 'product': 'CU_LT05.001'}, {'layer': 'SRB3', 'product': 'CU_LT05.001'}, {'layer': 'SRB4', 'product': 'CU_LT05.001'}, {'layer': 'SRB5', 'product': 'CU_LT05.001'}, {'layer': 'SRB7', 'product': 'CU_LT05.001'}, {'layer': 'SRCLOUDQA', 'product': 'CU_LT05.001'}], 'output': {'format': {'type': 'geotiff'}, 'projection': 'geographic'}, 'geo': {'type': 'FeatureCollection', 'features': [{'id': '0', 'type': 'Feature', 'properties': {'FID_1': 349.0, 'h': 18, 'v': 13}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-94.50856585825146, 35.330189994248286], [-92.84510497774326, 35.29742847715524], [-92.89937263707863, 33.95694461325491], [-94.53423443113005, 33.989190100776376], [-94.50856585825146, 35.330189994248286]]]}}]}}}
{'task_id': '9b037ad0-ce0e-4d7c-

KeyboardInterrupt: 

In [103]:
bundle = r.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json

files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values

destDir = os.path.join("D:\Zekun\Landsat_ARD\Landsat_ARD\wayne_ard_period2")#, #task_name)
if not os.path.exists(destDir) : os.makedirs(destDir)

for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), stream=True)           # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  # Parse the name from                                  Content-Disposition header 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir)) 

{'message': "ValueError('Geometry must be in a geographic projection',)"}
